In [ ]:
!pip install virtualenv
!mkdir my_virtualenv
%cd my_virtualenv
!virtualenv my_env
!source my_env/bin/activate

In [ ]:
!pip install langchain
!pip install pypdf
!pip install unstructured
!pip install sentence_transformers
!pip install pinecone-client
!pip install llama-cpp-python
!pip install huggingface_hub

In [24]:
from langchain.document_loaders import PyPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as pclang
from sentence_transformers import SentenceTransformer
from langchain.chains.question_answering import load_qa_chain
import pinecone
import os

In [5]:
# import pinecone
pc=pinecone.Pinecone(api_key='adf2f219-42d7-6be969dfe5dc', environment="gcp-starter")

In [14]:
!gdown "https://drive.google.com/uc?id=15hUEJQViQDxu_fnJeO_Og1hGqykCmJut&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=15hUEJQViQDxu_fnJeO_Og1hGqykCmJut&confirm=t
To: /content/my_virtualenv/The-Field-Guide-to-Data-Science.pdf
100% 30.3M/30.3M [00:01<00:00, 22.4MB/s]


In [15]:
#loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")
# loader = PyPDFLoader("/content/yolov7paper.pdf")
loader = PyPDFLoader("/content/my_virtualenv/The-Field-Guide-to-Data-Science.pdf")

In [16]:
data = loader.load()

In [17]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

In [18]:
docs=text_splitter.split_documents(data)


In [19]:
docs=docs[:100]


In [20]:
len(docs)


100

In [21]:
docs[11]

Document(page_content='replication also requires carefully \nbalanced processes and the right  \nset of environmental conditions.  \nIn the end, every instance may look \nsuper/f_icially diﬀerent, but the raw \nmaterials remain the same./guilsinglright.cap/guilsinglright.cap', metadata={'source': '/content/my_virtualenv/The-Field-Guide-to-Data-Science.pdf', 'page': 5})

In [22]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
index = pc.Index('langchain4march')

In [9]:
index

In [34]:
pc.describe_index("langchain4march")

{'dimension': 384,
 'host': 'langchain4march-fqrpmm1.svc.gcp-starter.pinecone.io',
 'metric': 'cosine',
 'name': 'langchain4march',
 'spec': {'pod': {'environment': 'gcp-starter',
                  'pod_type': 'starter',
                  'pods': 1,
                  'replicas': 1,
                  'shards': 1}},
 'status': {'ready': True, 'state': 'Ready'}}

In [26]:

index_name = "langchain4march" # put in the name of your pinecone index here
# pc=pinecone.Pinecone(api_key='5dc', environment="gcp-starter")


In [32]:
query_result = embeddings.embed_query("H")

In [33]:
print("Length", len(query_result))

Length 384


In [28]:
import os
os.environ['PINECONE_API_KEY'] = 'adfdfe5dc'

In [35]:
docsearch=pclang.from_texts([t.page_content for t in docs], embeddings, index_name=index_name)

In [36]:
query="What are examples of good data science teams?"

In [37]:
pc.list_indexes()

{'indexes': [{'dimension': 384,
              'host': 'langchain4march-fqrpmm1.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain4march',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [38]:
docs=docsearch.similarity_search(query)

In [39]:
docs

[Document(page_content='\x03›Data Science is a diﬀerent kind of team sport.   \nData Science teams need a broad view of the organization. Leaders \nmust be key advocates who meet with stakeholders to ferret out \nthe hardest challenges, locate the data, connect disparate parts of \nthe business, and gain widespread buy-in./guilsinglright.cap/guilsinglright.cap\n13 The Short Version 13'),
 Document(page_content='to achieve its objective – winning at the game of improved insights. \nYou need the three skill sets shown in /T_he Data Science Venn Diagram  \nto create a winning team in the world of Data Science.\nBuilding Data Science teams is diﬃcult. It requires an understanding \nof the types of personalities that make Data Science possible, as well \nas a willingness to establish a culture of innovation and curiosity in \nyour organization. You must also consider how to deploy the team and'),
 Document(page_content='Data Science teams. \n\x03›Data Science capabilities can be built over 

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir --verbose

In [42]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from huggingface_hub import hf_hub_download
from langchain.chains.question_answering import load_qa_chain

In [43]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

In [44]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format


In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [ ]:
n_gpu_layers = 40  # Change this value based on your model and your GPU VRAM pool.
n_batch = 256  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Loading model,
llm = LlamaCpp(
    model_path=model_path,
    max_tokens=256,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    n_ctx=1024,
    verbose=False,
)

In [ ]:
chain=load_qa_chain(llm, chain_type="stuff")

In [ ]:
query="YOLOv7 outperforms which models"
docs=docsearch.similarity_search(query)

In [ ]:
docs

In [ ]:
chain.run(input_documents=docs, question=query)

In [ ]:
query="YOLOv7 trained on which dataset"
docs=docsearch.similarity_search(query)

In [ ]:
chain.run(input_documents=docs, question=query)

In [ ]:
# import os
# from pinecone import Pinecone
# pc = Pinecone(api_key=os.environ.get('07ed3f1'))
# pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"), host=os.getenv("PINECONE_CONTROLLER_HOST"))

In [49]:
from langchain.llms import HuggingFaceHub

In [ ]:
from huggingface_hub import notebook_login

In [51]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [52]:
llm=HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512})

In [53]:
chain=load_qa_chain(llm, chain_type="stuff")

In [54]:
query="What are examples of good data science teams?"
docs=docsearch.similarity_search(query)

In [55]:
docs

[Document(page_content='\x03›Data Science is a diﬀerent kind of team sport.   \nData Science teams need a broad view of the organization. Leaders \nmust be key advocates who meet with stakeholders to ferret out \nthe hardest challenges, locate the data, connect disparate parts of \nthe business, and gain widespread buy-in./guilsinglright.cap/guilsinglright.cap\n13 The Short Version 13'),
 Document(page_content='to achieve its objective – winning at the game of improved insights. \nYou need the three skill sets shown in /T_he Data Science Venn Diagram  \nto create a winning team in the world of Data Science.\nBuilding Data Science teams is diﬃcult. It requires an understanding \nof the types of personalities that make Data Science possible, as well \nas a willingness to establish a culture of innovation and curiosity in \nyour organization. You must also consider how to deploy the team and'),
 Document(page_content='Data Science teams. \n\x03›Data Science capabilities can be built over 

In [56]:
chain.run(input_documents=docs, question=query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Data Science teams need a broad view of the organization. Leaders must be key advocates who meet with stakeholders to ferret out the hardest challenges, locate the data, connect disparate parts of the business, and gain widespread buy-in.'